In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing

In [4]:
#Load Base Dataser
demo = pd.read_csv('demographics.csv')
demo.head()

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 5 - 17,Female 18 - 59,...,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 5 - 17,Male 18 - 59,Male 60,Male total,m_other,m_unknown,Total
0,2019,Afghanistan,AFG,Afghanistan,AFG,299424,311116,250049,NaN,550020,...,294347,333850,275895,NaN,554205,68273,1526570,NaN,NaN,3000483
1,2019,Iran (Islamic Rep. of),IRN,Afghanistan,AFG,0,0,0,NaN,14,...,0,0,0,NaN,17,5,22,NaN,NaN,36
2,2019,Pakistan,PAK,Afghanistan,AFG,7521,8360,4890,NaN,15951,...,6787,8589,4322,NaN,12191,1996,33885,NaN,NaN,72191
3,2019,Algeria,DZA,Albania,ALB,0,0,0,NaN,0,...,0,0,0,NaN,11,0,11,NaN,NaN,11
4,2019,Egypt,EGY,Albania,ALB,0,0,0,NaN,0,...,0,5,0,NaN,0,0,5,NaN,NaN,5


In [5]:
#Read in GDP Dataset
gdp = pd.read_csv('gdp_pc.csv')
gdp_2019 = gdp[['Country Name','Country Code', 'Indicator Name', '2019']]
gdp_2019.head()

,Country Name,Country Code,Indicator Name,2019
0,Aruba,ABW,GDP per capita (current US$),NaN
1,Afghanistan,AFG,GDP per capita (current US$),507.103432
2,Angola,AGO,GDP per capita (current US$),2790.726615
3,Albania,ALB,GDP per capita (current US$),5353.244856
4,Andorra,AND,GDP per capita (current US$),40886.391160


In [6]:
#Merge GDP with Base Dataset
demo = demo.rename(columns={'Country of origin (ISO)':'Country Code'})
new_demo = pd.merge(demo,gdp_2019,on='Country Code')
#Filter Columns (Losing Demographic Specific Data)
demo_gdp = new_demo[['Country Name', 'Country of origin', 'Country Code', 'Country of asylum',
                    'Country of asylum (ISO)','Female total','Male total', 
                     'Total','2019']]
demo_gdp = demo_gdp.rename(columns={'2019':'gdp_per_cap'})
demo_gdp.head()

,Country Name,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap
0,Afghanistan,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432
1,Afghanistan,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432
2,Afghanistan,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432
3,Afghanistan,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432
4,Afghanistan,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432


In [7]:
#Read in GDP Per Capita Growth Rate
growth = pd.read_csv('gdp_growth.csv')
growth_2019 = growth[['Country Name','Country Code', 'Indicator Name', '2019']]
growth_2019.head()

,Country Name,Country Code,Indicator Name,2019
0,Aruba,ABW,GDP per capita growth (annual %),NaN
1,Afghanistan,AFG,GDP per capita growth (annual %),1.536912
2,Angola,AGO,GDP per capita growth (annual %),-3.795674
3,Albania,ALB,GDP per capita growth (annual %),2.676550
4,Andorra,AND,GDP per capita growth (annual %),1.669680


In [8]:
#Merge GDP Per Capita Growth Rate with Base Dataset
demo_gdp = pd.merge(demo_gdp,growth_2019,on='Country Code')
demo_gdp = demo_gdp[['Country Name_x', 'Country of origin', 'Country Code', 'Country of asylum',
                    'Country of asylum (ISO)','Female total','Male total', 'Total',
                      'gdp_per_cap','2019']]
demo_gdp = demo_gdp.rename(columns={'Country Name_x':'country_name', '2019':'gdp_growth_rate'})
demo_gdp.head()

,country_name,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap,gdp_growth_rate
0,Afghanistan,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432,1.536912
1,Afghanistan,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432,1.536912
2,Afghanistan,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432,1.536912
3,Afghanistan,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432,1.536912
4,Afghanistan,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432,1.536912


In [9]:
#Read in Inflation Dataset
inflation = pd.read_csv('inflation.csv')
inflation_2019 = inflation[['Country Name','Country Code', 'Indicator Name', '2019']]
inflation_2019.head()

,Country Name,Country Code,Indicator Name,2019
0,Aruba,ABW,"Inflation, GDP deflator (annual %)",NaN
1,Afghanistan,AFG,"Inflation, GDP deflator (annual %)",6.521480
2,Angola,AGO,"Inflation, GDP deflator (annual %)",27.229035
3,Albania,ALB,"Inflation, GDP deflator (annual %)",0.363243
4,Andorra,AND,"Inflation, GDP deflator (annual %)",1.507863


In [10]:
#Merge Inflation with Base Dataset
demo_gdp = pd.merge(demo_gdp,inflation_2019,on='Country Code')
demo_gdp = demo_gdp[['Country Name', 'Country of origin', 'Country Code', 'Country of asylum',
                    'Country of asylum (ISO)','Female total','Male total', 'Total',
                      'gdp_per_cap', 'gdp_growth_rate', '2019']]
demo_gdp = demo_gdp.rename(columns={'Country Name_x':'country_name', '2019':'inflation_rate'})
demo_gdp.head()

,Country Name,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap,gdp_growth_rate,inflation_rate
0,Afghanistan,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432,1.536912,6.52148
1,Afghanistan,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432,1.536912,6.52148
2,Afghanistan,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432,1.536912,6.52148
3,Afghanistan,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432,1.536912,6.52148
4,Afghanistan,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432,1.536912,6.52148


In [11]:
#Read in Latitude/Longitude Dataset
lat_long = pd.read_csv('lat_long.csv')
lat_long = lat_long.rename(columns={'name': 'Country Name'})
lat_long

,country,latitude,longitude,Country Name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [12]:
#Merge Latitude/Longitude with Base Dataset
demo_gdp1 = pd.merge(demo_gdp,lat_long,on='Country Name')
demo_gdp1 = demo_gdp1.rename(columns={'country': '2_letter_code'})
demo_gdp1.head()

,Country Name,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap,gdp_growth_rate,inflation_rate,2_letter_code,latitude,longitude
0,Afghanistan,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432,1.536912,6.52148,AF,33.93911,67.709953
1,Afghanistan,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432,1.536912,6.52148,AF,33.93911,67.709953
2,Afghanistan,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432,1.536912,6.52148,AF,33.93911,67.709953
3,Afghanistan,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432,1.536912,6.52148,AF,33.93911,67.709953
4,Afghanistan,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432,1.536912,6.52148,AF,33.93911,67.709953


In [13]:
#Remove Redundant Column
filt1 = demo_gdp1.drop("Country Name",axis=1)
filt1.head()

,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap,gdp_growth_rate,inflation_rate,2_letter_code,latitude,longitude
0,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432,1.536912,6.52148,AF,33.93911,67.709953
1,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432,1.536912,6.52148,AF,33.93911,67.709953
2,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432,1.536912,6.52148,AF,33.93911,67.709953
3,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432,1.536912,6.52148,AF,33.93911,67.709953
4,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432,1.536912,6.52148,AF,33.93911,67.709953


In [14]:
#Select Columns for GroupBy Totals
ref = filt1[['Country of origin','Country Code','Country of asylum', 'Country of asylum (ISO)', 'Female total', 'Male total','Total']]
ref.head()

,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total
0,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483
1,Afghanistan,AFG,Egypt,EGY,13,13,26
2,Afghanistan,AFG,Argentina,ARG,5,7,12
3,Afghanistan,AFG,Armenia,ARM,0,5,5
4,Afghanistan,AFG,Australia,AUS,0,0,11585


In [15]:
#Groupby Totals
grouped_ref = ref.groupby("Country of origin").sum()
grouped_ref = grouped_ref.reset_index()
grouped_ref = grouped_ref.rename(columns={"Country of origin":"origin_country","Female total":
                           "female_total","Male total":"male_total","Total":"total"})
grouped_ref.head()

,origin_country,female_total,male_total,total
0,Afghanistan,2203060,2412457,5731617
1,Albania,3869,3476,15027
2,Algeria,1298,1640,4619
3,Angola,12150,11963,26414
4,Antigua and Barbuda,0,0,117


In [16]:
#Reset for GroupBy Merge
gdp_2019 = gdp_2019.rename(columns={"Country Name":"origin_country"})
growth_2019 = growth_2019.rename(columns={"Country Name":"origin_country"})
inflation_2019 = inflation_2019.rename(columns={"Country Name":"origin_country"})
lat_long = lat_long.rename(columns={"Country Name":"origin_country"})                           
# grouped_1 = pd.merge(grouped_ref,gdp2019,on='Country Code')

In [17]:
#Merge All - GroupBy
grouped_0 = pd.merge(grouped_ref,lat_long,on='origin_country')
grouped_0 = grouped_0.rename(columns={'country':'2_letter_code'})

grouped_1 = pd.merge(grouped_0,gdp_2019,on='origin_country')
grouped_1 = grouped_1.rename(columns={'2019':'gdp_per_cap','Country Code':'3_letter_code'})
grouped_1 = grouped_1[['origin_country','2_letter_code','3_letter_code','female_total',
                       'male_total','total', 'latitude','longitude',
                      'gdp_per_cap']]


grouped_2 = pd.merge(grouped_1,growth_2019,on='origin_country')
grouped_2 = grouped_2.drop("Country Code",axis=1)
grouped_2 = grouped_2.drop("Indicator Name",axis=1)
grouped_2 = grouped_2.rename(columns={'2019':'gdp_per_cap_growth_rate'})


grouped_3 = pd.merge(grouped_2,inflation_2019,on='origin_country')
grouped_3 = grouped_3.drop("Country Code",axis=1)
grouped_3 = grouped_3.drop("Indicator Name",axis=1)
grouped_3 = grouped_3.rename(columns={'2019':'inflation_rate'})

combined_refugees = grouped_3


# grouped_4 = pd.merge(grouped_3,lat_long,on='origin_country')

In [18]:
#Combined Dataset - Without Asylum Specific Breakdowns and Demographic Breakdown
combined_refugees.head()

,origin_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate
0,Afghanistan,AF,AFG,2203060,2412457,5731617,33.939110,67.709953,507.103432,1.536912,6.521480
1,Albania,AL,ALB,3869,3476,15027,41.153332,20.168331,5353.244856,2.676550,0.363243
2,Algeria,DZ,DZA,1298,1640,4619,28.033886,1.659626,3973.964072,-1.130692,-0.910066
3,Angola,AO,AGO,12150,11963,26414,-11.202692,17.873887,2790.726615,-3.795674,27.229035
4,Antigua and Barbuda,AG,ATG,0,0,117,17.060816,-61.796428,17112.821130,3.762049,-1.081851


In [19]:
#Combined Dataset - With Asylum Specific Breakdowns and without Demographic Breakdown
paired_refugees = filt1
paired_refugees.head()

,Country of origin,Country Code,Country of asylum,Country of asylum (ISO),Female total,Male total,Total,gdp_per_cap,gdp_growth_rate,inflation_rate,2_letter_code,latitude,longitude
0,Afghanistan,AFG,Afghanistan,AFG,1473913,1526570,3000483,507.103432,1.536912,6.52148,AF,33.93911,67.709953
1,Afghanistan,AFG,Egypt,EGY,13,13,26,507.103432,1.536912,6.52148,AF,33.93911,67.709953
2,Afghanistan,AFG,Argentina,ARG,5,7,12,507.103432,1.536912,6.52148,AF,33.93911,67.709953
3,Afghanistan,AFG,Armenia,ARM,0,5,5,507.103432,1.536912,6.52148,AF,33.93911,67.709953
4,Afghanistan,AFG,Australia,AUS,0,0,11585,507.103432,1.536912,6.52148,AF,33.93911,67.709953


In [22]:
#Read In Population Dataset
pop = pd.read_csv("pop.csv")
pop2019 = pop[['Country Name','Country Code','2019']]
pop2019 = pop2019.rename(columns = {'Country Name':'origin_country', 'Country Code':'3_letter_code',
                                   '2019':'total_population'})
pop2019.head()

,origin_country,3_letter_code,total_population
0,Aruba,ABW,106314.0
1,Afghanistan,AFG,38041754.0
2,Angola,AGO,31825295.0
3,Albania,ALB,2854191.0
4,Andorra,AND,77142.0


In [23]:
scaled_ref = pd.merge(combined_refugees,pop2019, on = "3_letter_code")
scaled_ref = scaled_ref.drop("origin_country_y",axis=1)
scaled_ref = scaled_ref.rename(columns = {'origin_country_x':'origin_country'})
scaled_ref.head()

,origin_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population
0,Afghanistan,AF,AFG,2203060,2412457,5731617,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0
1,Albania,AL,ALB,3869,3476,15027,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0
2,Algeria,DZ,DZA,1298,1640,4619,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0
3,Angola,AO,AGO,12150,11963,26414,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0
4,Antigua and Barbuda,AG,ATG,0,0,117,17.060816,-61.796428,17112.821130,3.762049,-1.081851,97118.0


In [24]:
scaled_ref['female_ref_to_total_pop_ratio'] = scaled_ref['female_total']/scaled_ref['total_population']
scaled_ref['male_ref_to_total_pop_ratio'] = scaled_ref['male_total']/scaled_ref['total_population']
scaled_ref['total_ref_to_total_pop_ratio'] = scaled_ref['female_total']/scaled_ref['total_population']

In [25]:
scaled_ref

,origin_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population,female_ref_to_total_pop_ratio,male_ref_to_total_pop_ratio,total_ref_to_total_pop_ratio
0,Afghanistan,AF,AFG,2203060,2412457,5731617,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0,5.791163e-02,0.063416,5.791163e-02
1,Albania,AL,ALB,3869,3476,15027,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0,1.355550e-03,0.001218,1.355550e-03
2,Algeria,DZ,DZA,1298,1640,4619,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0,3.014885e-05,0.000038,3.014885e-05
3,Angola,AO,AGO,12150,11963,26414,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0,3.817718e-04,0.000376,3.817718e-04
4,Antigua and Barbuda,AG,ATG,0,0,117,17.060816,-61.796428,17112.821130,3.762049,-1.081851,97118.0,0.000000e+00,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,United Arab Emirates,AE,ARE,22,17,155,23.424076,53.847818,43103.323060,0.225275,-1.899928,9770529.0,2.251669e-06,0.000002,2.251669e-06
142,Uruguay,UY,URY,0,0,15,-32.522779,-55.765835,16190.126960,-0.137889,7.667916,3461734.0,0.000000e+00,0.000000,0.000000e+00
143,Uzbekistan,UZ,UZB,204,233,3219,41.377491,64.585262,1724.841134,3.600396,19.233621,33580650.0,6.074927e-06,0.000007,6.074927e-06
144,Zambia,ZM,ZMB,11,0,264,-13.133897,27.849332,1305.063254,-1.450264,7.632867,17861030.0,6.158659e-07,0.000000,6.158659e-07


In [26]:
#Export Data
# combined_refugees.to_csv("combined_refugees.csv")
# paired_refugees.to_csv("paired_refugees.csv")

In [27]:
#Make Table for Countries of Asylum

In [28]:
#Asylum Totals
grouped_asy = ref.groupby("Country of asylum").sum()
grouped_asy = grouped_asy.reset_index()
grouped_asy = grouped_asy.rename(columns={"Country of asylum":"asylum_country","Female total":
                           "female_total","Male total":"male_total","Total":"total"})
grouped_asy.head()

,asylum_country,female_total,male_total,total
0,Afghanistan,1512219,1560455,3072674
1,Albania,38,166,204
2,Algeria,105,185,290
3,Angola,0,0,2432
4,Argentina,961,1338,2299


In [29]:
#Reset for GroupBy Merge
a_gdp_2019 = gdp_2019.rename(columns={"origin_country":"asylum_country"})
a_growth_2019 = growth_2019.rename(columns={"origin_country":"asylum_country"})
a_inflation_2019 = inflation_2019.rename(columns={"origin_country":"asylum_country"})
a_lat_long = lat_long.rename(columns={"origin_country":"asylum_country"})                           
# grouped_1 = pd.merge(grouped_ref,gdp2019,on='Country Code')

In [30]:
#Merge All - GroupBy
asylum_0 = pd.merge(grouped_asy,a_lat_long,on='asylum_country')
asylum_0 = asylum_0.rename(columns={'country':'2_letter_code'})

asylum_1 = pd.merge(asylum_0,a_gdp_2019,on='asylum_country')
asylum_1 = asylum_1.rename(columns={'2019':'gdp_per_cap','Country Code':'3_letter_code'})
asylum_1 = asylum_1[['asylum_country','2_letter_code','3_letter_code','female_total',
                       'male_total','total', 'latitude','longitude',
                      'gdp_per_cap']]


asylum_2 = pd.merge(asylum_1,a_growth_2019,on='asylum_country')
asylum_2 = asylum_2.drop("Country Code",axis=1)
asylum_2 = asylum_2.drop("Indicator Name",axis=1)
asylum_2 = asylum_2.rename(columns={'2019':'gdp_per_cap_growth_rate'})


asylum_3 = pd.merge(asylum_2,a_inflation_2019,on='asylum_country')
asylum_3 = asylum_3.drop("Country Code",axis=1)
asylum_3 = asylum_3.drop("Indicator Name",axis=1)
asylum_3 = asylum_3.rename(columns={'2019':'inflation_rate'})

combined_asylum = asylum_3


In [31]:
combined_asylum.head()

,asylum_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate
0,Afghanistan,AF,AFG,1512219,1560455,3072674,33.939110,67.709953,507.103432,1.536912,6.521480
1,Albania,AL,ALB,38,166,204,41.153332,20.168331,5353.244856,2.676550,0.363243
2,Algeria,DZ,DZA,105,185,290,28.033886,1.659626,3973.964072,-1.130692,-0.910066
3,Angola,AO,AGO,0,0,2432,-11.202692,17.873887,2790.726615,-3.795674,27.229035
4,Argentina,AR,ARG,961,1338,2299,-38.416097,-63.616672,9912.281809,-3.055855,50.622568


In [32]:
scaled_asylum = pd.merge(combined_asylum,pop2019, on = "3_letter_code")
scaled_asylum = scaled_asylum.drop("origin_country",axis=1)
scaled_asylum.head()

,asylum_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population
0,Afghanistan,AF,AFG,1512219,1560455,3072674,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0
1,Albania,AL,ALB,38,166,204,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0
2,Algeria,DZ,DZA,105,185,290,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0
3,Angola,AO,AGO,0,0,2432,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0
4,Argentina,AR,ARG,961,1338,2299,-38.416097,-63.616672,9912.281809,-3.055855,50.622568,44938712.0


In [33]:
scaled_asylum['female_ref_to_total_pop_ratio'] = scaled_asylum['female_total']/scaled_asylum['total_population']
scaled_asylum['male_ref_to_total_pop_ratio'] = scaled_asylum['male_total']/scaled_asylum['total_population']
scaled_asylum['total_ref_to_total_pop_ratio'] = scaled_asylum['female_total']/scaled_asylum['total_population']

In [34]:
scaled_asylum

,asylum_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population,female_ref_to_total_pop_ratio,male_ref_to_total_pop_ratio,total_ref_to_total_pop_ratio
0,Afghanistan,AF,AFG,1512219,1560455,3072674,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0,3.975156e-02,4.101953e-02,3.975156e-02
1,Albania,AL,ALB,38,166,204,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0,1.331376e-05,5.816009e-05,1.331376e-05
2,Algeria,DZ,DZA,105,185,290,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0,2.438851e-06,4.297024e-06,2.438851e-06
3,Angola,AO,AGO,0,0,2432,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0,0.000000e+00,0.000000e+00,0.000000e+00
4,Argentina,AR,ARG,961,1338,2299,-38.416097,-63.616672,9912.281809,-3.055855,50.622568,44938712.0,2.138468e-05,2.977388e-05,2.138468e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,United Arab Emirates,AE,ARE,495,462,957,23.424076,53.847818,43103.323060,0.225275,-1.899928,9770529.0,5.066256e-05,4.728505e-05,5.066256e-05
129,Uruguay,UY,URY,147,199,346,-32.522779,-55.765835,16190.126960,-0.137889,7.667916,3461734.0,4.246427e-05,5.748564e-05,4.246427e-05
130,Uzbekistan,UZ,UZB,6,7,13,41.377491,64.585262,1724.841134,3.600396,19.233621,33580650.0,1.786743e-07,2.084534e-07,1.786743e-07
131,Zambia,ZM,ZMB,15355,17759,33114,-13.133897,27.849332,1305.063254,-1.450264,7.632867,17861030.0,8.596929e-04,9.942876e-04,8.596929e-04


In [35]:
#Export Data
#scaled_asylum.to_csv('scaled_asylum.csv')

In [36]:
scaled_ref.head()

,origin_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population,female_ref_to_total_pop_ratio,male_ref_to_total_pop_ratio,total_ref_to_total_pop_ratio
0,Afghanistan,AF,AFG,2203060,2412457,5731617,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0,0.057912,0.063416,0.057912
1,Albania,AL,ALB,3869,3476,15027,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0,0.001356,0.001218,0.001356
2,Algeria,DZ,DZA,1298,1640,4619,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0,0.000030,0.000038,0.000030
3,Angola,AO,AGO,12150,11963,26414,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0,0.000382,0.000376,0.000382
4,Antigua and Barbuda,AG,ATG,0,0,117,17.060816,-61.796428,17112.821130,3.762049,-1.081851,97118.0,0.000000,0.000000,0.000000


In [37]:
scaled_asylum.head()

,asylum_country,2_letter_code,3_letter_code,female_total,male_total,total,latitude,longitude,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population,female_ref_to_total_pop_ratio,male_ref_to_total_pop_ratio,total_ref_to_total_pop_ratio
0,Afghanistan,AF,AFG,1512219,1560455,3072674,33.939110,67.709953,507.103432,1.536912,6.521480,38041754.0,0.039752,0.041020,0.039752
1,Albania,AL,ALB,38,166,204,41.153332,20.168331,5353.244856,2.676550,0.363243,2854191.0,0.000013,0.000058,0.000013
2,Algeria,DZ,DZA,105,185,290,28.033886,1.659626,3973.964072,-1.130692,-0.910066,43053054.0,0.000002,0.000004,0.000002
3,Angola,AO,AGO,0,0,2432,-11.202692,17.873887,2790.726615,-3.795674,27.229035,31825295.0,0.000000,0.000000,0.000000
4,Argentina,AR,ARG,961,1338,2299,-38.416097,-63.616672,9912.281809,-3.055855,50.622568,44938712.0,0.000021,0.000030,0.000021


In [1]:
#len(scaled_ref)

In [2]:
#len(scaled_asylum)

In [52]:
aggregate = scaled_ref[['origin_country','2_letter_code','3_letter_code','female_total','male_total','total']]

In [61]:
aggregate.head()

,asylum_country,2_letter_code,3_letter_code,female_total_out,male_total_out,total_out,female_total_in,male_total_in,total_in,latitude,...,gdp_per_cap,gdp_per_cap_growth_rate,inflation_rate,total_population,female_ref_to_total_pop_ratio,male_ref_to_total_pop_ratio,total_ref_to_total_pop_ratio,female_out_in_ratio,male_out_in_ratio,total_out_in_ratio
0,Afghanistan,AF,AFG,2203060,2412457,5731617,1512219,1560455,3072674,33.939110,...,507.103432,1.536912,6.521480,38041754.0,0.039752,0.041020,0.039752,1.456839,1.545996,1.865351
1,Albania,AL,ALB,3869,3476,15027,38,166,204,41.153332,...,5353.244856,2.676550,0.363243,2854191.0,0.000013,0.000058,0.000013,101.815789,20.939759,73.661765
2,Algeria,DZ,DZA,1298,1640,4619,105,185,290,28.033886,...,3973.964072,-1.130692,-0.910066,43053054.0,0.000002,0.000004,0.000002,12.361905,8.864865,15.927586
3,Angola,AO,AGO,12150,11963,26414,0,0,2432,-11.202692,...,2790.726615,-3.795674,27.229035,31825295.0,0.000000,0.000000,0.000000,inf,inf,10.861020
4,Argentina,AR,ARG,13,7,104,961,1338,2299,-38.416097,...,9912.281809,-3.055855,50.622568,44938712.0,0.000021,0.000030,0.000021,0.013528,0.005232,0.045237


In [54]:
aggregate = aggregate.rename(columns={'origin_country':'asylum_country',
                         'female_total':'female_total_out',
                         'male_total':'male_total_out',
                         'total':'total_out'})

In [55]:
aggregate = pd.merge(aggregate,scaled_asylum,on='asylum_country')

In [56]:
aggregate = aggregate.drop('2_letter_code_y',axis=1)
aggregate = aggregate.drop('3_letter_code_y',axis=1)

In [58]:
aggregate = aggregate.rename(columns={
    '2_letter_code_x':'2_letter_code',
    '3_letter_code_x':'3_letter_code',
    'female_total':'female_total_in',
    'male_total':'male_total_in',
    'total':'total_in' 
})

In [60]:
aggregate['female_out_in_ratio'] = aggregate['female_total_out']/aggregate['female_total_in']
aggregate['male_out_in_ratio'] = aggregate['male_total_out']/aggregate['male_total_in']
aggregate['total_out_in_ratio'] = aggregate['total_out']/aggregate['total_in']

In [62]:
#Export Aggregate
aggregate.to_csv('aggregate.csv')